In [33]:
import numpy as np
import pandas as pd
import seaborn as sb
% matplotlib inline
from datetime import datetime, timedelta

from bs4 import BeautifulSoup as bs
import requests

In [25]:
# Scrape data

years = range(2002, 2018)
letter = range(0, 26)
URL = "https://services.datasport.com/{}/lauf/lamara/ALFAA.HTM"

rankings = {}
headers = {}
for year in years:
    r = requests.get(URL.format(year))
    soup = bs(r.text, 'html.parser')
    title = soup.find('title').getText()
    if(title != 'Adresse nicht vorhanden / The address is not available'):
        data = soup.findAll('font', {'size':'2'})

        print(year)
        headers[str(year)] = data[0].getText()
        rankings[str(year)] = data[1].getText().split('¦ \r\n')


2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017


In [132]:
def parse_columns(ranking):
    column_structure = {}
    column_mask = np.ones(np.array(list(ranking[0])).shape, dtype=bool)
    spaces_mask = np.ones(np.array(list(ranking[0])).shape)*32
    for line in ranking[:-1]:
        cur_line = np.array(list(line)).view(np.uint32)
        
        try:
            column_mask = np.logical_and(column_mask, np.equal(cur_line, spaces_mask))
        except:
            pass
        
    inside_space_col = False
    delimiters = []
    for i, is_space_col in enumerate(list(column_mask)):
        if is_space_col:
            if not inside_space_col:
                inside_space_col = True
                delimiters.append(i)
                
        elif inside_space_col:
            inside_space_col = False
            
    return delimiters
    

def separate_cols(ranking, header):
    delimiters = parse_columns(ranking)
    
    start = 0
    cols = {} 
    unnamed = 0
    for delimiter in delimiters:
        key = header[start:delimiter+1].strip()
        if key == '':
            key = unnamed
            unnamed += 1
        cols[key] = (start, delimiter+1)
        start = delimiter+1
    
    start = 0
    result = {}
    for i, line in enumerate(ranking):
        for col in cols:
            start, end = cols[col]
            if col in result:
                result[col].append(line[start:end].strip())
            else:
                result[col] = [line[start:end].strip()]
    return pd.DataFrame(result, columns=result.keys())

dfs = {}
for year in rankings:
    dfs[year] = separate_cols(rankings[year], headers[year])

In [139]:
dfs['2017'].head()

,catégorie,0,2,3,lieu,1,rang,doss,an,overall,temps,moyenne,retard,équipe,nom
0,10-H30,diplôme,video,2569.,N-Oteren,foto,676.,(16798),1981,10-Hom,"1:15.48,7",7.34,"45.43,3",,Aarskog Kay-Morten
1,21-H20,diplôme,video,2358.,Ecublens VD,foto,484.,(3616),1994,21-Hom,"2:03.37,1",5.51,"53.40,3",,Abadie Théo
2,21-H50,diplôme,video,1179.,St-Légier-La Chiésaz,foto,139.,(4078),1966,21-Hom,"1:45.25,0",4.59,"23.51,0",,Abaidia Jilani
3,21-D40,diplôme,video,337.,St-Légier,foto,77.,(4076),1972,21-Fem,"1:53.23,1",5.22,"30.40,6",,Abaidia Sandrine
4,10-H20,diplôme,video,1684.,Renens VD,foto,428.,(12280),1991,10-Hom,"53.23,8",5.20,"23.59,5",,Abawi Khaled
